In [ ]:
# Check GPU availability
!nvidia-smi

# Install system dependencies
!apt-get update
!apt-get install -y libopenexr-dev

Thu Jun 19 20:26:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
!mkdir -p "/content/drive/MyDrive/ESLAM_outputs"

Mounted at /content/drive


In [ ]:
# Clone ESLAM repository
!git clone https://github.com/idiap/ESLAM.git
%cd ESLAM

# Check structure
!ls -la

fatal: destination path 'ESLAM' already exists and is not an empty directory.
/content/ESLAM
total 68
drwxr-xr-x 6 root root  4096 Jun 19 20:19 .
drwxr-xr-x 1 root root  4096 Jun 19 20:27 ..
drwxr-xr-x 5 root root  4096 Jun 19 20:19 configs
-rw-r--r-- 1 root root  5671 Jun 19 20:19 environment.yaml
drwxr-xr-x 8 root root  4096 Jun 19 20:19 .git
-rw-r--r-- 1 root root    65 Jun 19 20:19 .gitignore
-rw-r--r-- 1 root root 11342 Jun 19 20:19 LICENSE
-rw-r--r-- 1 root root  6492 Jun 19 20:19 README.md
-rw-r--r-- 1 root root  2682 Jun 19 20:19 run.py
drwxr-xr-x 2 root root  4096 Jun 19 20:19 scripts
drwxr-xr-x 5 root root  4096 Jun 19 20:19 src
-rw-r--r-- 1 root root  5863 Jun 19 20:19 visualizer.py


In [ ]:
# Install PyTorch and essential packages
!pip install torch torchvision torchaudio
!pip install opencv-python pyyaml tqdm matplotlib plyfile trimesh imageio
!pip install numpy scipy scikit-learn pillow

# Verify installations
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 129.0 MB/s eta 0:00:00
PyTorch version: 2.6.0+cu124
CUDA available: True


In [ ]:
# Find where Open3D is used
!grep -r "import open3d\|from open3d" . --include="*.py"

./src/tools/eval_recon.py:import open3d as o3d
./src/tools/visualizer_util.py:import open3d as o3d
./src/utils/Mesher.py:import open3d as o3d


In [ ]:
# First, let's see what's in the main run.py to understand the flow
!head -30 run.py

# This file is a part of ESLAM.
#
# ESLAM is a NeRF-based SLAM system. It utilizes Neural Radiance Fields (NeRF)
# to perform Simultaneous Localization and Mapping (SLAM) in real-time.
# This software is the implementation of the paper "ESLAM: Efficient Dense SLAM
# System Based on Hybrid Representation of Signed Distance Fields" by
# Mohammad Mahdi Johari, Camilla Carta, and Francois Fleuret.
#
# Copyright 2023 ams-OSRAM AG
#
# Author: Mohammad Mahdi Johari <mohammad.johari@idiap.ch>
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions a

In [ ]:
# Check if Mesher.py is critical for core SLAM
!grep -n "Mesher\|mesher" run.py
!grep -n "Mesher\|mesher" src/*.py

src/ESLAM.py:55:from src.utils.Mesher import Mesher
src/ESLAM.py:140:        self.mesher = Mesher(cfg, args, self)
src/Mapper.py:74:        self.mesher = eslam.mesher
src/Mapper.py:464:                self.mesher.get_mesh(mesh_out_file, all_planes, self.decoders, self.keyframe_dict, self.device)
src/Mapper.py:473:                self.mesher.get_mesh(mesh_out_file, all_planes, self.decoders, self.keyframe_dict, self.device)
src/Tracker.py:71:        self.mesher = eslam.mesher


In [ ]:
# Create modified versions of the problematic files
# 1. Replace Mesher.py with a no-op version
%%writefile src/utils/Mesher_no_op.py
"""
No-op Mesher that skips mesh extraction to avoid Open3D dependency
"""

class Mesher:
    def __init__(self, *args, **kwargs):
        print("Mesher initialized in no-op mode (skipping mesh extraction)")

    def update_mesh(self, *args, **kwargs):
        print("Skipping mesh update (no-op mode)")
        return None

    def extract_mesh(self, *args, **kwargs):
        print("Skipping mesh extraction (no-op mode)")
        return None

    def get_mesh(self, *args, **kwargs):
        print("Skipping get_mesh (no-op mode)")
        return None

    def __getattr__(self, name):
        def mock_method(*args, **kwargs):
            print(f"Skipping Mesher method: {name}")
            return None
        return mock_method

Writing src/utils/Mesher_no_op.py


In [ ]:
# 2. Backup original Mesher and replace with no-op version
!cp src/utils/Mesher.py src/utils/Mesher_original.py
!cp src/utils/Mesher_no_op.py src/utils/Mesher.py

In [ ]:
# 3. Let's look at the main ESLAM class to see what we need to modify
!find src -name "*.py" -exec grep -l "class.*SLAM\|class.*Slam" {} \;

src/tools/visualizer_util.py
src/ESLAM.py


In [ ]:
# Check the main ESLAM implementation
!ls src/
!find src -name "*.py" | head -10

common.py  ESLAM.py	Mapper.py  tools       utils
config.py  __init__.py	networks   Tracker.py
src/networks/decoders.py
src/networks/config.py
src/networks/__init__.py
src/common.py
src/tools/cull_mesh.py
src/tools/eval_recon.py
src/tools/visualizer_util.py
src/tools/eval_ate.py
src/config.py
src/ESLAM.py


In [ ]:
# First check the config files
!cat configs/Replica/room0.yaml

inherit_from: configs/Replica/replica.yaml
mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
data:
  input_folder: Datasets/Replica/room0
  output: output/Replica/room0

In [ ]:
# Download the dataset
!bash scripts/download_replica.sh

Streaming output truncated to the last 5000 lines.
  inflating: Replica/office4/results/depth001392.png  
  inflating: Replica/office4/results/depth000557.png  
  inflating: Replica/office4/results/frame001118.jpg  
  inflating: Replica/office4/results/frame001777.jpg  
  inflating: Replica/office4/results/frame000324.jpg  
  inflating: Replica/office4/results/depth000959.png  
  inflating: Replica/office4/results/depth001962.png  
  inflating: Replica/office4/results/depth000894.png  
  inflating: Replica/office4/results/depth000857.png  
  inflating: Replica/office4/results/frame001503.jpg  
  inflating: Replica/office4/results/frame001177.jpg  
  inflating: Replica/office4/results/depth000026.png  
  inflating: Replica/office4/results/depth000561.png  
  inflating: Replica/office4/results/frame000664.jpg  
  inflating: Replica/office4/results/frame001675.jpg  
  inflating: Replica/office4/results/depth001708.png  
  inflating: Replica/office4/results/depth001488.png  
  inflating: R

In [ ]:
# Check the parent config that room0.yaml inherits from
!cat configs/Replica/replica.yaml

dataset: 'replica'
meshing:
  resolution: 0.01 # Increase this number to speed up meshing algorithm
  eval_rec: True
tracking:
  ignore_edge_W: 75
  ignore_edge_H: 75
  lr_T: 0.002
  lr_R: 0.001
  pixels: 2000
  iters: 8
mapping:
  pixels: 4000
  iters: 15
cam:
  H: 680
  W: 1200
  fx: 600.0
  fy: 600.0
  cx: 599.5
  cy: 339.5
  png_depth_scale: 6553.5 #for depth image in png format
  crop_edge: 0
rendering:
  n_stratified: 32
  n_importance: 8
  learnable_beta: True

In [ ]:
# Check if ESLAM.yaml exists (the main config)
!cat configs/ESLAM.yaml

scale: 1
verbose: False
device: "cuda:0"
keyframe_device: "cuda:0" ## Change this to "cpu" to reduce GPU memory usage
planes_res:
  coarse: 0.24
  fine: 0.06
  bound_dividable: 0.24
c_planes_res:
  coarse: 0.24
  fine: 0.03
meshing:
  level_set: 0
  resolution: 0.01 # Increase this number to speed up meshing algorithm
  eval_rec: False
  mesh_bound_scale: 1.02
tracking:
  ignore_edge_W: 75
  ignore_edge_H: 75
  vis_freq: 4000
  vis_inside_freq: 400
  const_speed_assumption: True
  no_vis_on_first_frame: True
  gt_camera: False
  lr_T: 0.001
  lr_R: 0.001
  pixels: 2000
  iters: 8
  w_sdf_fs: 10
  w_sdf_center: 200
  w_sdf_tail: 50
  w_depth: 1
  w_color: 5
mapping:
  every_frame: 4
  joint_opt: True
  joint_opt_cam_lr: 0.001
  no_vis_on_first_frame: True
  no_mesh_on_first_frame: True
  no_log_on_first_frame: True
  vis_freq: 4000
  vis_inside_freq: 400 #each iteration
  mesh_freq: 4000
  ckpt_freq: 500
  keyframe_every: 4
  mapping_window_size: 20
  keyframe_selection_method: 'overlap

In [ ]:
# Create a no-visualization config
%%writefile configs/Replica/room0_no_vis.yaml
inherit_from: configs/Replica/replica.yaml

# Override visualization and meshing settings
tracking:
  ignore_edge_W: 75
  ignore_edge_H: 75
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  const_speed_assumption: True
  no_vis_on_first_frame: True
  gt_camera: False
  lr_T: 0.001
  lr_R: 0.001
  pixels: 2000
  iters: 8
  w_sdf_fs: 10
  w_sdf_center: 200
  w_sdf_tail: 50
  w_depth: 1
  w_color: 5

mapping:
  every_frame: 4
  joint_opt: True
  joint_opt_cam_lr: 0.001
  no_vis_on_first_frame: True
  no_mesh_on_first_frame: True
  no_log_on_first_frame: True
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  mesh_freq: -1  # Disable mesh extraction
  ckpt_freq: 500
  keyframe_every: 4
  mapping_window_size: 20
  keyframe_selection_method: 'overlap'
  lr_first_factor: 5
  lr_factor: 1
  pixels: 4000
  iters_first: 1000
  iters: 15
  w_sdf_fs: 5
  w_sdf_center: 200
  w_sdf_tail: 10
  w_depth: 0.1
  w_color: 5
  lr:
    decoders_lr: 0.001
    planes_lr: 0.005
    c_planes_lr: 0.005

# Override meshing settings
meshing:
  level_set: 0
  resolution: 0.01
  eval_rec: False  # Disable reconstruction evaluation
  mesh_bound_scale: 1.02

# Specify the data and output paths
mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]

data:
  input_folder: Datasets/Replica/room0

output: output/Replica/room0_no_vis

Writing configs/Replica/room0_no_vis.yaml


In [ ]:
# Create the no-op Mesher
%%writefile src/utils/Mesher_no_op.py
"""
No-op Mesher that skips mesh extraction to avoid Open3D dependency
"""

class Mesher:
    def __init__(self, *args, **kwargs):
        print("Mesher initialized in no-op mode (skipping mesh extraction)")

    def update_mesh(self, *args, **kwargs):
        print("Skipping mesh update (no-op mode)")
        return None

    def extract_mesh(self, *args, **kwargs):
        print("Skipping mesh extraction (no-op mode)")
        return None

    def get_mesh(self, *args, **kwargs):
        print("Skipping get_mesh (no-op mode)")
        return None

    def save_mesh(self, *args, **kwargs):
        print("Skipping save_mesh (no-op mode)")
        return None

    def __getattr__(self, name):
        def mock_method(*args, **kwargs):
            print(f"Skipping Mesher method: {name}")
            return None
        return mock_method

# Backup and replace the original Mesher
!cp src/utils/Mesher.py src/utils/Mesher_original.py
!cp src/utils/Mesher_no_op.py src/utils/Mesher.py

Overwriting src/utils/Mesher_no_op.py


In [ ]:
# Create and load the Open3D mock
%%writefile mock_open3d.py
class MockOpen3D:
    def __getattr__(self, name):
        def mock_function(*args, **kwargs):
            print(f"Skipping Open3D function: {name}")
            return self
        return mock_function

import sys
sys.modules['open3d'] = MockOpen3D()

# Load the mock
exec(open('mock_open3d.py').read())

Writing mock_open3d.py


In [ ]:
# Install PyTorch3D - this might take a few minutes
!pip install pytorch3d

ERROR: Could not find a version that satisfies the requirement pytorch3d (from versions: none)
ERROR: No matching distribution found for pytorch3d


In [ ]:
# First, let's see exactly what PyTorch3D functions are used
!grep -r "from pytorch3d" . --include="*.py"

./src/common.py:from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix


In [ ]:
# Create a minimal PyTorch3D replacement
%%writefile pytorch3d_minimal.py
"""
Minimal PyTorch3D replacement for ESLAM
Only implements the functions actually used by ESLAM
"""
import torch
import torch.nn.functional as F

def matrix_to_quaternion(matrix):
    """
    Convert rotation matrix to quaternion
    Simplified implementation - may not be as robust as PyTorch3D
    """
    # This is a simplified version - you might need a more robust implementation
    # For now, let's create a basic version that won't crash
    batch_size = matrix.shape[0]
    # Return identity quaternions [w, x, y, z] format
    return torch.tensor([[1.0, 0.0, 0.0, 0.0]], device=matrix.device).repeat(batch_size, 1)

def quaternion_to_matrix(quaternion):
    """
    Convert quaternion to rotation matrix
    Simplified implementation
    """
    batch_size = quaternion.shape[0]
    # Return identity matrices
    return torch.eye(3, device=quaternion.device).unsqueeze(0).repeat(batch_size, 1, 1)

# Create the transforms module structure
class MockTransforms:
    @staticmethod
    def matrix_to_quaternion(matrix):
        return matrix_to_quaternion(matrix)

    @staticmethod
    def quaternion_to_matrix(quaternion):
        return quaternion_to_matrix(quaternion)

# Mock the pytorch3d module structure
import sys
sys.modules['pytorch3d'] = type('MockPyTorch3D', (), {})()
sys.modules['pytorch3d.transforms'] = MockTransforms()

Overwriting pytorch3d_minimal.py


In [ ]:
# Create our PyTorch3D replacement
exec(open('pytorch3d_minimal.py').read())

# Now modify src/common.py to use our replacement
# First, let's see the current import
!grep -A 5 -B 5 "from pytorch3d" src/common.py

    # See the License for the specific language governing permissions and
    # limitations under the License.

import numpy as np
import torch
from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix

def as_intrinsics_matrix(intrinsics):
    """
    Get matrix representation of intrinsics.



In [ ]:
# Create a patched version of common.py
!cp src/common.py src/common_original.py

# Replace the pytorch3d import with our local functions
!sed -i 's/from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix/# from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix/' src/common.py

In [ ]:
# Add our replacement functions to the top of common.py
%%writefile temp_functions.py
import torch
import torch.nn.functional as F

def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)

    batch_size = matrix.shape[0]
    trace = matrix[:, 0, 0] + matrix[:, 1, 1] + matrix[:, 2, 2]

    quaternions = torch.zeros(batch_size, 4, device=matrix.device, dtype=matrix.dtype)

    # Simple conversion (can be improved for robustness)
    mask = trace > 0
    if mask.any():
        s = torch.sqrt(trace[mask] + 1.0) * 2
        quaternions[mask, 0] = 0.25 * s  # w
        quaternions[mask, 1] = (matrix[mask, 2, 1] - matrix[mask, 1, 2]) / s  # x
        quaternions[mask, 2] = (matrix[mask, 0, 2] - matrix[mask, 2, 0]) / s  # y
        quaternions[mask, 3] = (matrix[mask, 1, 0] - matrix[mask, 0, 1]) / s  # z

    # Handle negative trace case with identity quaternion
    quaternions[~mask, 0] = 1.0  # w = 1 for identity

    return quaternions

def quaternion_to_matrix(quaternion):
    """Convert quaternion to rotation matrix"""
    if quaternion.dim() == 1:
        quaternion = quaternion.unsqueeze(0)

    # Normalize quaternion
    quaternion = F.normalize(quaternion, dim=1)

    w, x, y, z = quaternion[:, 0], quaternion[:, 1], quaternion[:, 2], quaternion[:, 3]

    # Compute rotation matrix
    xx, yy, zz = x*x, y*y, z*z
    xy, xz, yz = x*y, x*z, y*z
    wx, wy, wz = w*x, w*y, w*z

    batch_size = quaternion.shape[0]
    matrix = torch.zeros(batch_size, 3, 3, device=quaternion.device, dtype=quaternion.dtype)

    matrix[:, 0, 0] = 1 - 2*(yy + zz)
    matrix[:, 0, 1] = 2*(xy - wz)
    matrix[:, 0, 2] = 2*(xz + wy)
    matrix[:, 1, 0] = 2*(xy + wz)
    matrix[:, 1, 1] = 1 - 2*(xx + zz)
    matrix[:, 1, 2] = 2*(yz - wx)
    matrix[:, 2, 0] = 2*(xz - wy)
    matrix[:, 2, 1] = 2*(yz + wx)
    matrix[:, 2, 2] = 1 - 2*(xx + yy)

    return matrix

Writing temp_functions.py


In [ ]:
# Now prepend these functions to common.py
!python -c "
with open('temp_functions.py', 'r') as f:
    functions = f.read()

with open('src/common.py', 'r') as f:
    original = f.read()

# Add our functions after the existing imports but before the pytorch3d import line
lines = original.split('\n')
new_lines = []
pytorch3d_found = False

for line in lines:
    if 'from pytorch3d.transforms import' in line and not pytorch3d_found:
        # Replace the pytorch3d import with our functions
        new_lines.append('# ' + line + ' # Replaced with local implementation')
        new_lines.append('')
        new_lines.extend(functions.split('\n'))
        new_lines.append('')
        pytorch3d_found = True
    else:
        new_lines.append(line)

with open('src/common.py', 'w') as f:
    f.write('\n'.join(new_lines))

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
!ls -la

total 84
drwxr-xr-x 7 root root  4096 Jun 19 20:56 .
drwxr-xr-x 1 root root  4096 Jun 19 20:27 ..
drwxr-xr-x 5 root root  4096 Jun 19 20:19 configs
drwxr-xr-x 3 root root  4096 Jun 19 20:44 Datasets
-rw-r--r-- 1 root root  5671 Jun 19 20:19 environment.yaml
drwxr-xr-x 8 root root  4096 Jun 19 20:19 .git
-rw-r--r-- 1 root root    65 Jun 19 20:19 .gitignore
-rw-r--r-- 1 root root 11342 Jun 19 20:19 LICENSE
-rw-r--r-- 1 root root   305 Jun 19 20:48 mock_open3d.py
-rw-r--r-- 1 root root  1350 Jun 19 20:55 pytorch3d_minimal.py
-rw-r--r-- 1 root root  6492 Jun 19 20:19 README.md
-rw-r--r-- 1 root root  2682 Jun 19 20:19 run.py
drwxr-xr-x 2 root root  4096 Jun 19 20:19 scripts
drwxr-xr-x 6 root root  4096 Jun 19 20:56 src
-rw-r--r-- 1 root root  1934 Jun 19 20:56 temp_functions.py
-rw-r--r-- 1 root root  5863 Jun 19 20:19 visualizer.py


In [ ]:
# Clean up temp file
!rm temp_functions.py

# Verify the modification worked
!grep -A 10 -B 5 "matrix_to_quaternion" src/common.py | head -20

    # See the License for the specific language governing permissions and
    # limitations under the License.

import numpy as np
import torch
# # # from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix # Replaced with local implementation # Replaced with local implementation

import torch
import torch.nn.functional as F

def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)
    
    batch_size = matrix.shape[0]
    trace = matrix[:, 0, 0] + matrix[:, 1, 1] + matrix[:, 2, 2]
    
    quaternions = torch.zeros(batch_size, 4, device=matrix.device, dtype=matrix.dtype)
    


In [ ]:
!pip install colorama

In [ ]:
pip install colorama


In [ ]:
pip install mapper

In [ ]:
# Create the no-op Mesher
%%writefile src/utils/Mesher_no_op.py
"""
No-op Mesher that skips mesh extraction to avoid Open3D dependency
"""

class Mesher:
    def __init__(self, *args, **kwargs):
        print("Mesher initialized in no-op mode (skipping mesh extraction)")

    def update_mesh(self, *args, **kwargs):
        print("Skipping mesh update (no-op mode)")
        return None

    def extract_mesh(self, *args, **kwargs):
        print("Skipping mesh extraction (no-op mode)")
        return None

    def get_mesh(self, *args, **kwargs):
        print("Skipping get_mesh (no-op mode)")
        return None

    def save_mesh(self, *args, **kwargs):
        print("Skipping save_mesh (no-op mode)")
        return None

    def __getattr__(self, name):
        def mock_method(*args, **kwargs):
            print(f"Skipping Mesher method: {name}")
            return None
        return mock_method

Overwriting src/utils/Mesher_no_op.py


In [ ]:
!cp src/utils/Mesher.py src/utils/Mesher_original.py
!cp src/utils/Mesher_no_op.py src/utils/Mesher.py

In [ ]:
%%writefile configs/Replica/room0_no_vis.yaml
inherit_from: configs/Replica/replica.yaml

# Override visualization and meshing settings
tracking:
  ignore_edge_W: 75
  ignore_edge_H: 75
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  const_speed_assumption: True
  no_vis_on_first_frame: True
  gt_camera: False
  lr_T: 0.001
  lr_R: 0.001
  pixels: 2000
  iters: 8
  w_sdf_fs: 10
  w_sdf_center: 200
  w_sdf_tail: 50
  w_depth: 1
  w_color: 5

mapping:
  every_frame: 4
  joint_opt: True
  joint_opt_cam_lr: 0.001
  no_vis_on_first_frame: True
  no_mesh_on_first_frame: True
  no_log_on_first_frame: True
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  mesh_freq: -1  # Disable mesh extraction
  ckpt_freq: 500
  keyframe_every: 4
  mapping_window_size: 20
  keyframe_selection_method: 'overlap'
  lr_first_factor: 5
  lr_factor: 1
  pixels: 4000
  iters_first: 1000
  iters: 15
  w_sdf_fs: 5
  w_sdf_center: 200
  w_sdf_tail: 10
  w_depth: 0.1
  w_color: 5
  lr:
    decoders_lr: 0.001
    planes_lr: 0.005
    c_planes_lr: 0.005

# Override meshing settings
meshing:
  level_set: 0
  resolution: 0.01
  eval_rec: False  # Disable reconstruction evaluation
  mesh_bound_scale: 1.02

# Specify the data and output paths
mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]

data:
  input_folder: Datasets/Replica/room0

output: output/Replica/room0_no_vis

Overwriting configs/Replica/room0_no_vis.yaml


In [ ]:
# First, let's create the temp file again
%%writefile temp_functions.py
import torch
import torch.nn.functional as F

def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)

    batch_size = matrix.shape[0]
    trace = matrix[:, 0, 0] + matrix[:, 1, 1] + matrix[:, 2, 2]

    quaternions = torch.zeros(batch_size, 4, device=matrix.device, dtype=matrix.dtype)

    # Simple conversion (can be improved for robustness)
    mask = trace > 0
    if mask.any():
        s = torch.sqrt(trace[mask] + 1.0) * 2
        quaternions[mask, 0] = 0.25 * s  # w
        quaternions[mask, 1] = (matrix[mask, 2, 1] - matrix[mask, 1, 2]) / s  # x
        quaternions[mask, 2] = (matrix[mask, 0, 2] - matrix[mask, 2, 0]) / s  # y
        quaternions[mask, 3] = (matrix[mask, 1, 0] - matrix[mask, 0, 1]) / s  # z

    # Handle negative trace case with identity quaternion
    quaternions[~mask, 0] = 1.0  # w = 1 for identity

    return quaternions

def quaternion_to_matrix(quaternion):
    """Convert quaternion to rotation matrix"""
    if quaternion.dim() == 1:
        quaternion = quaternion.unsqueeze(0)

    # Normalize quaternion
    quaternion = F.normalize(quaternion, dim=1)

    w, x, y, z = quaternion[:, 0], quaternion[:, 1], quaternion[:, 2], quaternion[:, 3]

    # Compute rotation matrix
    xx, yy, zz = x*x, y*y, z*z
    xy, xz, yz = x*y, x*z, y*z
    wx, wy, wz = w*x, w*y, w*z

    batch_size = quaternion.shape[0]
    matrix = torch.zeros(batch_size, 3, 3, device=quaternion.device, dtype=quaternion.dtype)

    matrix[:, 0, 0] = 1 - 2*(yy + zz)
    matrix[:, 0, 1] = 2*(xy - wz)
    matrix[:, 0, 2] = 2*(xz + wy)
    matrix[:, 1, 0] = 2*(xy + wz)
    matrix[:, 1, 1] = 1 - 2*(xx + zz)
    matrix[:, 1, 2] = 2*(yz - wx)
    matrix[:, 2, 0] = 2*(xz - wy)
    matrix[:, 2, 1] = 2*(yz + wx)
    matrix[:, 2, 2] = 1 - 2*(xx + yy)

    return matrix

Overwriting temp_functions.py


In [ ]:
# Create a Python script to do the modification
%%writefile fix_common.py
with open('temp_functions.py', 'r') as f:
    functions = f.read()

with open('src/common.py', 'r') as f:
    original = f.read()

# Simple replacement
new_content = original.replace(
    'from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix',
    '# from pytorch3d.transforms import matrix_to_quaternion, quaternion_to_matrix\n\n' + functions
)

with open('src/common.py', 'w') as f:
    f.write(new_content)

print("Modified src/common.py successfully")

Writing fix_common.py


In [ ]:
# Run the fix script
!python fix_common.py

Modified src/common.py successfully


In [ ]:
# Clean up
!rm temp_functions.py fix_common.py

In [ ]:
# Verify it worked
!grep -A 5 "def matrix_to_quaternion" src/common.py

def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)
    
    batch_size = matrix.shape[0]
--
def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)
    
    batch_size = matrix.shape[0]
--
def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)
    
    batch_size = matrix.shape[0]
--
def matrix_to_quaternion(matrix):
    """Convert rotation matrix to quaternion - simplified version"""
    if matrix.dim() == 2:
        matrix = matrix.unsqueeze(0)
    
    batch_size = matrix.shape[0]


In [ ]:
!pip install colorama


In [ ]:
!python -W ignore run.py configs/Replica/room0_no_vis.yaml

Traceback (most recent call last):
  File "/content/ESLAM/run.py", line 45, in <module>
    from src.ESLAM import ESLAM
  File "/content/ESLAM/src/ESLAM.py", line 51, in <module>
    from src.Mapper import Mapper
  File "/content/ESLAM/src/Mapper.py", line 53, in <module>
    from src.utils.Frame_Visualizer import Frame_Visualizer
  File "/content/ESLAM/src/utils/Frame_Visualizer.py", line 45, in <module>
    import matplotlib.pyplot as plt
  File "/usr/local/lib/python3.13/site-packages/matplotlib/__init__.py", line 1296, in <module>
    rcParams['backend'] = os.environ.get('MPLBACKEND')
    ~~~~~~~~^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/matplotlib/__init__.py", line 771, in __setitem__
    raise ValueError(f"Key {key}: {ve}") from None
ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg

In [ ]:
# Run ESLAM with our PyTorch3D replacement
!python -W ignore run.py configs/Replica/room0_no_vis.yaml

Traceback (most recent call last):
  File "/content/ESLAM/run.py", line 45, in <module>
    from src.ESLAM import ESLAM
  File "/content/ESLAM/src/ESLAM.py", line 51, in <module>
    from src.Mapper import Mapper
  File "/content/ESLAM/src/Mapper.py", line 53, in <module>
    from src.utils.Frame_Visualizer import Frame_Visualizer
  File "/content/ESLAM/src/utils/Frame_Visualizer.py", line 45, in <module>
    import matplotlib.pyplot as plt
  File "/usr/local/lib/python3.13/site-packages/matplotlib/__init__.py", line 1296, in <module>
    rcParams['backend'] = os.environ.get('MPLBACKEND')
    ~~~~~~~~^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/matplotlib/__init__.py", line 771, in __setitem__
    raise ValueError(f"Key {key}: {ve}") from None
ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg

In [ ]:
# Fix matplotlib backend issue
import os
os.environ['MPLBACKEND'] = 'Agg'  # Use non-interactive backend

# Also set it in matplotlib directly
import matplotlib
matplotlib.use('Agg')

In [ ]:
!python -W ignore run.py configs/Replica/room0_no_vis.yaml

Traceback (most recent call last):
  File "/content/ESLAM/run.py", line 65, in <module>
    main()
    ~~~~^^
  File "/content/ESLAM/run.py", line 60, in main
    eslam = ESLAM(cfg, args)
  File "/content/ESLAM/src/ESLAM.py", line 80, in __init__
    self.output = cfg['data']['output']
                  ~~~~~~~~~~~^^^^^^^^^^
KeyError: 'output'


In [ ]:
# Create the correct config matching the original structure
%%writefile configs/Replica/room0_no_vis.yaml
inherit_from: configs/Replica/replica.yaml

mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  mesh_freq: -1  # Disable mesh extraction

tracking:
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization

meshing:
  eval_rec: false  # Disable reconstruction evaluation

data:
  input_folder: Datasets/Replica/room0
  output: output/Replica/room0_no_vis

Overwriting configs/Replica/room0_no_vis.yaml


In [ ]:
!MPLBACKEND=Agg python -W ignore run.py configs/Replica/room0_no_vis.yaml

Traceback (most recent call last):
  File "/content/ESLAM/run.py", line 65, in <module>
    main()
    ~~~~^^
  File "/content/ESLAM/run.py", line 60, in main
    eslam = ESLAM(cfg, args)
  File "/content/ESLAM/src/ESLAM.py", line 80, in __init__
    self.output = cfg['data']['output']
                  ~~~~~~~~~~~^^^^^^^^^^
KeyError: 'output'


In [ ]:
# Let's see the exact structure expected by looking at the original config
!cat configs/Replica/room0.yaml

inherit_from: configs/Replica/replica.yaml
mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
data:
  input_folder: Datasets/Replica/room0
  output: output/Replica/room0

In [ ]:
# Let's also check the ESLAM.py code to see what it expects
!grep -A 5 -B 5 "cfg\['data'\]\['output'\]" src/ESLAM.py

        self.device = cfg['device']
        self.dataset = cfg['dataset']
        self.truncation = cfg['model']['truncation']

        if args.output is None:
            self.output = cfg['data']['output']
        else:
            self.output = args.output
        self.ckptsdir = os.path.join(self.output, 'ckpts')
        os.makedirs(self.output, exist_ok=True)
        os.makedirs(self.ckptsdir, exist_ok=True)


In [ ]:
!MPLBACKEND=Agg python -W ignore run.py configs/Replica/room0_no_vis.yaml

Mesher initialized in no-op mode (skipping mesh extraction)
INFO: The output folder is output/Replica/room0_no_vis
INFO: The GT, generated and residual depth/color images can be found under output/Replica/room0_no_vis/tracking_vis/ and output/Replica/room0_no_vis/mapping_vis/
INFO: The mesh can be found under output/Replica/room0_no_vis/mesh/
INFO: The checkpoint can be found under output/Replica/room0_no_vis/ckpt/
Tracking Frame 5:   0% 5/2000 [00:58<6:38:38, 11.99s/it]Skipping get_mesh (no-op mode)
Process Process-2:
Traceback (most recent call last):
  File "/usr/local/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/usr/local/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ESLAM/src/ESLAM.py", line 263, in mapping
    self.mapper.run()
    ~~~~~~~~~~~~~~~^^
  File "/content/ESLAM/src/Mapper.py", line 465,

In [ ]:
# Add frame limit to the config
%%writefile configs/Replica/room0_no_vis.yaml
inherit_from: configs/Replica/replica.yaml

mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  mesh_freq: -1  # Disable mesh extraction

tracking:
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization

meshing:
  eval_rec: false  # Disable reconstruction evaluation

data:
  input_folder: Datasets/Replica/room0
  output: output/Replica/room0_no_vis
  end_frame: 500  # Process only first 500 frames

# Alternative way to limit frames
max_frames: 500

Overwriting configs/Replica/room0_no_vis.yaml


In [ ]:
# Create a better config that more completely disables meshing
%%writefile configs/Replica/room0_no_vis.yaml
inherit_from: configs/Replica/replica.yaml

mapping:
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  mesh_freq: -1  # Disable mesh extraction
  no_mesh_on_first_frame: true
  ckpt_freq: 100  # Save checkpoints less frequently

tracking:
  vis_freq: -1  # Disable visualization
  vis_inside_freq: -1  # Disable inside visualization
  no_vis_on_first_frame: true

meshing:
  eval_rec: false  # Disable reconstruction evaluation

data:
  input_folder: Datasets/Replica/room0
  output: output/Replica/room0_no_vis

# Frame limiting - try different parameter names
end_frame: 500
max_frame: 500
n_frames: 500

Overwriting configs/Replica/room0_no_vis.yaml


In [ ]:
# Check if outputs exist locally
!ls -la output/
!find . -name "*.txt" -o -name "*.log" -o -name "*.ply" -o -name "*.json" | grep -v __pycache__ | head -10

# Check what's in the output directory
!ls -la output/Replica/room0_no_vis/ 2>/dev/null || echo "Output directory doesn't exist yet"

total 12
drwxr-xr-x 3 root root 4096 Jun 19 21:30 .
drwxr-xr-x 8 root root 4096 Jun 19 21:30 ..
drwxr-xr-x 3 root root 4096 Jun 19 21:30 Replica
./Datasets/Replica/office4/traj.txt
./Datasets/Replica/office3_mesh.ply
./Datasets/Replica/cam_params.json
./Datasets/Replica/office0/traj.txt
./Datasets/Replica/room1_mesh.ply
./Datasets/Replica/office4_mesh.ply
./Datasets/Replica/room2/traj.txt
./Datasets/Replica/office3/traj.txt
./Datasets/Replica/room0_mesh.ply
./Datasets/Replica/office2_mesh.ply
total 24
drwxr-xr-x 6 root root 4096 Jun 19 21:30 .
drwxr-xr-x 3 root root 4096 Jun 19 21:30 ..
drwxr-xr-x 2 root root 4096 Jun 19 21:30 ckpts
drwxr-xr-x 2 root root 4096 Jun 19 21:32 mapping_vis
drwxr-xr-x 2 root root 4096 Jun 19 21:30 mesh
drwxr-xr-x 2 root root 4096 Jun 19 21:33 tracking_vis


In [ ]:
# Check what's in each output directory
!ls -la output/Replica/room0_no_vis/ckpts/
!ls -la output/Replica/room0_no_vis/mesh/
!ls -la output/Replica/room0_no_vis/tracking_vis/
!ls -la output/Replica/room0_no_vis/mapping_vis/

total 8
drwxr-xr-x 2 root root 4096 Jun 19 21:30 .
drwxr-xr-x 6 root root 4096 Jun 19 21:30 ..
total 8
drwxr-xr-x 2 root root 4096 Jun 19 21:30 .
drwxr-xr-x 6 root root 4096 Jun 19 21:30 ..
total 8400
drwxr-xr-x 2 root root   4096 Jun 19 21:33 .
drwxr-xr-x 6 root root   4096 Jun 19 21:30 ..
-rw-r--r-- 1 root root 141457 Jun 19 21:39 00001_0000.jpg
-rw-r--r-- 1 root root 139041 Jun 19 21:39 00001_0001.jpg
-rw-r--r-- 1 root root 135443 Jun 19 21:39 00001_0002.jpg
-rw-r--r-- 1 root root 131281 Jun 19 21:39 00001_0003.jpg
-rw-r--r-- 1 root root 131069 Jun 19 21:39 00001_0004.jpg
-rw-r--r-- 1 root root 131678 Jun 19 21:39 00001_0005.jpg
-rw-r--r-- 1 root root 130916 Jun 19 21:39 00001_0006.jpg
-rw-r--r-- 1 root root 129984 Jun 19 21:39 00001_0007.jpg
-rw-r--r-- 1 root root 133399 Jun 19 21:39 00002_0000.jpg
-rw-r--r-- 1 root root 133528 Jun 19 21:39 00002_0001.jpg
-rw-r--r-- 1 root root 131467 Jun 19 21:39 00002_0002.jpg
-rw-r--r-- 1 root root 131822 Jun 19 21:39 00002_0003.jpg
-rw-r--r-- 1

In [ ]:
# Look for trajectory files that ESLAM should have generated
!find output/Replica/room0_no_vis/ -name "*traj*" -o -name "*pose*" -o -name "*camera*"
!find output/Replica/room0_no_vis/ -name "*.txt" -o -name "*.log"

In [ ]:
# Stop the current run (Ctrl+C or Runtime -> Interrupt execution)
# Then run with the new config
!MPLBACKEND=Agg python -W ignore run.py configs/Replica/room0_no_vis.yaml

Mesher initialized in no-op mode (skipping mesh extraction)
INFO: The output folder is output/Replica/room0_no_vis
INFO: The GT, generated and residual depth/color images can be found under output/Replica/room0_no_vis/tracking_vis/ and output/Replica/room0_no_vis/mapping_vis/
INFO: The mesh can be found under output/Replica/room0_no_vis/mesh/
INFO: The checkpoint can be found under output/Replica/room0_no_vis/ckpt/
Tracking Frame 5:   0% 5/2000 [01:00<6:51:14, 12.37s/it]Traceback (most recent call last):
  File "/content/ESLAM/run.py", line 65, in <module>
    main()
  File "/content/ESLAM/run.py", line 62, in main
    eslam.run()
  File "/content/ESLAM/src/ESLAM.py", line 279, in run
    p.join()
  File "/usr/local/lib/python3.13/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/local/lib/python3.13/multiprocessing/popen_fork.py", line 44, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/local/lib/python3.13/m

In [ ]:
# Create a comprehensive config for evaluation with 500 frames
%%writefile configs/Replica/room0_eval_500.yaml
inherit_from: configs/Replica/replica.yaml

# Frame limiting - try multiple parameter names to ensure it works
end_frame: 500
max_frame: 500
n_frames: 500

# Core SLAM settings - keep essential functionality
tracking:
  ignore_edge_W: 75
  ignore_edge_H: 75
  vis_freq: 500  # Visualize only at the end
  vis_inside_freq: -1  # Disable frequent inside visualization
  const_speed_assumption: True
  no_vis_on_first_frame: True
  gt_camera: False
  lr_T: 0.001
  lr_R: 0.001
  pixels: 2000
  iters: 8
  w_sdf_fs: 10
  w_sdf_center: 200
  w_sdf_tail: 50
  w_depth: 1
  w_color: 5

mapping:
  every_frame: 4
  joint_opt: True
  joint_opt_cam_lr: 0.001
  no_vis_on_first_frame: True
  no_mesh_on_first_frame: True
  no_log_on_first_frame: False  # Enable logging for evaluation
  vis_freq: 500  # Visualize only at the end
  vis_inside_freq: -1  # Disable frequent inside visualization
  mesh_freq: 500  # Extract mesh only at the end for evaluation
  ckpt_freq: 250  # Save checkpoints twice during run
  keyframe_every: 4
  mapping_window_size: 20
  keyframe_selection_method: 'overlap'
  lr_first_factor: 5
  lr_factor: 1
  pixels: 4000
  iters_first: 1000
  iters: 15
  w_sdf_fs: 5
  w_sdf_center: 200
  w_sdf_tail: 10
  w_depth: 0.1
  w_color: 5
  lr:
    decoders_lr: 0.001
    planes_lr: 0.005
    c_planes_lr: 0.005
  bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]
  marching_cubes_bound: [[-1.9,7.9],[-2.2,4.5],[-2.5,2.3]]

# Meshing settings - enable for reconstruction evaluation
meshing:
  level_set: 0
  resolution: 0.02  # Slightly coarser for speed
  eval_rec: True  # ENABLE reconstruction evaluation
  mesh_bound_scale: 1.02

# Data and output paths
data:
  input_folder: Datasets/Replica/room0
  output: /content/drive/MyDrive/ESLAM_outputs/room0_eval_500

# Additional settings to ensure outputs are saved
save_rendered_image: True
save_depth_image: True
eval_rendering: True
verbose: True

Writing configs/Replica/room0_eval_500.yaml


In [ ]:
# Clear any existing outputs and start fresh
!rm -rf output/Replica/room0_no_vis
!mkdir -p /content/drive/MyDrive/ESLAM_outputs/room0_eval_500

# Set matplotlib backend and run ESLAM
!MPLBACKEND=Agg python -W ignore run.py configs/Replica/room0_eval_500.yaml

Mesher initialized in no-op mode (skipping mesh extraction)
INFO: The output folder is /content/drive/MyDrive/ESLAM_outputs/room0_eval_500
INFO: The GT, generated and residual depth/color images can be found under /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/tracking_vis/ and /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/mapping_vis/
INFO: The mesh can be found under /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/mesh/
INFO: The checkpoint can be found under /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ckpt/

Mapping Frame  0

Saved checkpoints at /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ckpts/00000.tar
Tracking: update the parameters from mapping

Tracking Frame  0


Tracking Frame  1


Tracking Frame  2


Tracking Frame  3


Tracking Frame  4


Mapping Frame  4

Tracking: update the parameters from mapping

Tracking Frame  5


Tracking Frame  6


Tracking Frame  7


Tracking Frame  8


Mapping Frame  8

Tracking: update the parameters from mapping


In [ ]:
# Check all output directories and file counts
!find /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ -type f | wc -l
!find /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ -type f -name "*.jpg" | wc -l
!find /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ -type f -name "*.txt" | head -10
!find /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ -type f -name "*.log" | head -10

25
23


In [ ]:
# Look for trajectory/pose data files (the key files for ATE evaluation)
!ls -la /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/
!ls -la /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ckpts/

# Check if there are any text files with trajectory data
!find /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ -name "*.txt" -exec ls -la {} \;

total 16
drwx------ 2 root root 4096 Jun 19 21:52 ckpts
drwx------ 2 root root 4096 Jun 19 21:52 mapping_vis
drwx------ 2 root root 4096 Jun 19 21:48 mesh
drwx------ 2 root root 4096 Jun 19 21:51 tracking_vis
total 538
-rw------- 1 root root 269225 Jun 19 21:49 00000.tar
-rw------- 1 root root 280793 Jun 19 21:52 00500.tar


In [ ]:
import torch
import numpy as np
import os
from scipy.spatial.transform import Rotation

def extract_trajectory():
    """Extract camera trajectory from ESLAM checkpoint for ATE evaluation"""

    ckpt_path = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ckpts/00500.tar"
    output_dir = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/"

    if not os.path.exists(ckpt_path):
        print(f"Checkpoint file not found: {ckpt_path}")
        return False

    print(f"Loading checkpoint from {ckpt_path}")
    try:
        checkpoint = torch.load(ckpt_path, map_location='cpu')

        print("\nAvailable keys in checkpoint:")
        for key in checkpoint.keys():
            data = checkpoint[key]
            print(f"  {key}: {type(data)}")
            if hasattr(data, 'shape'):
                print(f"    Shape: {data.shape}")
            elif isinstance(data, (list, tuple)):
                print(f"    Length: {len(data)}")
                if len(data) > 0:
                    print(f"    First element type: {type(data[0])}")
                    if hasattr(data[0], 'shape'):
                        print(f"    First element shape: {data[0].shape}")

        # Look for camera poses - try different possible keys
        pose_keys = ['estimate_c2w_list', 'c2w_list', 'poses', 'camera_poses', 'trajectory']
        poses = None

        for key in pose_keys:
            if key in checkpoint:
                poses = checkpoint[key]
                print(f"\nFound camera poses under key '{key}': {len(poses)} poses")
                break

        if poses is None:
            print("\nNo camera poses found. Available keys:")
            print(list(checkpoint.keys()))
            return False

        # Convert poses to TUM format
        traj_file = os.path.join(output_dir, "estimated_trajectory.txt")
        print(f"\nConverting {len(poses)} poses to TUM format...")

        with open(traj_file, 'w') as f:
            for i, pose in enumerate(poses):
                if isinstance(pose, torch.Tensor):
                    pose = pose.cpu().numpy()

                # Ensure pose is 4x4 matrix
                if pose.shape != (4, 4):
                    print(f"Warning: Expected 4x4 pose matrix, got {pose.shape}")
                    continue

                # Extract translation and rotation
                translation = pose[:3, 3]
                rotation_matrix = pose[:3, :3]

                # Convert rotation matrix to quaternion
                try:
                    r = Rotation.from_matrix(rotation_matrix)
                    quat = r.as_quat()  # [x, y, z, w] format

                    # Write in TUM format: timestamp tx ty tz qx qy qz qw
                    f.write(f"{i:06d} {translation[0]:.6f} {translation[1]:.6f} {translation[2]:.6f} "
                           f"{quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}\n")
                except Exception as e:
                    print(f"Error processing pose {i}: {e}")
                    continue

        print(f"Saved trajectory to {traj_file}")

        # Also save some statistics
        stats_file = os.path.join(output_dir, "trajectory_stats.txt")
        with open(stats_file, 'w') as f:
            f.write(f"ESLAM Trajectory Statistics\n")
            f.write(f"===========================\n")
            f.write(f"Total poses: {len(poses)}\n")
            f.write(f"First pose translation: {poses[0][:3, 3] if len(poses) > 0 else 'N/A'}\n")
            f.write(f"Last pose translation: {poses[-1][:3, 3] if len(poses) > 0 else 'N/A'}\n")

            # Calculate trajectory length
            if len(poses) > 1:
                total_distance = 0
                for i in range(1, len(poses)):
                    if isinstance(poses[i], torch.Tensor):
                        curr_pos = poses[i][:3, 3].cpu().numpy()
                        prev_pos = poses[i-1][:3, 3].cpu().numpy()
                    else:
                        curr_pos = poses[i][:3, 3]
                        prev_pos = poses[i-1][:3, 3]
                    total_distance += np.linalg.norm(curr_pos - prev_pos)
                f.write(f"Total trajectory length: {total_distance:.3f} meters\n")

        return True

    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return False

def compare_with_ground_truth():
    """Compare extracted trajectory with ground truth"""

    gt_file = "Datasets/Replica/room0/traj.txt"
    est_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt"

    if not os.path.exists(gt_file):
        print(f"Ground truth file not found: {gt_file}")
        return

    if not os.path.exists(est_file):
        print(f"Estimated trajectory file not found: {est_file}")
        return

    print("\nComparing trajectories...")
    print(f"Ground truth: {gt_file}")
    print(f"Estimated: {est_file}")

    # Count lines in both files
    with open(gt_file, 'r') as f:
        gt_lines = len(f.readlines())

    with open(est_file, 'r') as f:
        est_lines = len(f.readlines())

    print(f"Ground truth poses: {gt_lines}")
    print(f"Estimated poses: {est_lines}")

    # Show first few lines of each
    print("\nFirst 3 lines of ground truth:")
    with open(gt_file, 'r') as f:
        for i, line in enumerate(f):
            if i < 3:
                print(f"  {line.strip()}")

    print("\nFirst 3 lines of estimated:")
    with open(est_file, 'r') as f:
        for i, line in enumerate(f):
            if i < 3:
                print(f"  {line.strip()}")

if __name__ == "__main__":
    # Extract trajectory
    success = extract_trajectory()

    if success:
        # Compare with ground truth
        compare_with_ground_truth()

        print("\n" + "="*50)
        print("EVALUATION READY!")
        print("="*50)
        print("Next steps:")
        print("1. Install evo: !pip install evo")
        print("2. Run ATE evaluation:")
        print("   !evo_ape tum Datasets/Replica/room0/traj.txt \\")
        print("     /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt \\")
        print("     --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip")
    else:
        print("Failed to extract trajectory. Please check the checkpoint file.")

Loading checkpoint from /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ckpts/00500.tar

Available keys in checkpoint:
  decoder_state_dict: <class 'collections.OrderedDict'>
  gt_c2w_list: <class 'torch.Tensor'>
    Shape: torch.Size([2000, 4, 4])
  estimate_c2w_list: <class 'torch.Tensor'>
    Shape: torch.Size([2000, 4, 4])
  keyframe_list: <class 'list'>
    Length: 126
    First element type: <class 'torch.Tensor'>
    First element shape: torch.Size([])
  idx: <class 'torch.Tensor'>
    Shape: torch.Size([])

Found camera poses under key 'estimate_c2w_list': 2000 poses

Converting 2000 poses to TUM format...
Error processing pose 501: Non-positive determinant (left-handed or null coordinate frame) in rotation matrix 0: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]].
Error processing pose 502: Non-positive determinant (left-handed or null coordinate frame) in rotation matrix 0: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]].
Error processing pose 503: Non-positive determinant (left-handed or null

In [ ]:
# Install evaluation tool
!pip install evo

# Run ATE evaluation
!evo_ape tum Datasets/Replica/room0/traj.txt /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
Initialized new /root/.evo/settings.json
[ERROR] TUM trajectory files must have 8 entries per row and no trailing delimiter at the end of the rows (space)


In [ ]:
# First, let's check the ground truth trajectory format
!head -5 Datasets/Replica/room0/traj.txt
!wc -l Datasets/Replica/room0/traj.txt

-3.205696220032456800e-01 4.480551946958012954e-01 -8.345547674986967257e-01 3.452987416406734678e+00 9.472249560947475500e-01 1.516354520391845484e-01 -2.824386167580063001e-01 4.546110134135947223e-01 1.078977932490423164e-16 -8.810523436158487209e-01 -4.730187816662466682e-01 5.936285447159415085e-01 0.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e+00
-3.139329310152697561e-01 4.530657137828867920e-01 -8.343725629588769621e-01 3.457151553373337016e+00 9.494451615676188228e-01 1.498056477907042938e-01 -2.758843110179438618e-01 4.697096078660035756e-01 1.076219806983754285e-16 -8.788001632250705963e-01 -4.771899759169185296e-01 5.942743770991206764e-01 0.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e+00
-3.084751309215140735e-01 4.567669288580877285e-01 -8.343902362231387926e-01 3.461394428741041196e+00 9.512324077758046448e-01 1.481249345883572377e-01 -2.705843863755633683e-01 4.8418577244

In [ ]:
# Check our generated trajectory format
!head -5 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt 2>/dev/null || echo "File not found"
!wc -l /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt 2>/dev/null || echo "File not found"

000000 3.452987 0.454611 0.593629 0.299185 0.417108 0.697356 0.500203
000001 3.458086 0.458754 0.589913 0.301152 0.413567 0.695761 0.504169
000002 3.463855 0.465824 0.588772 0.302009 0.410939 0.694982 0.506873
000003 3.467487 0.479921 0.589067 0.302542 0.409459 0.694072 0.508995
000004 3.471281 0.495672 0.591193 0.302290 0.408571 0.694056 0.509879
501 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation

def convert_replica_to_tum():
    """Convert Replica ground truth trajectory from 4x4 matrices to TUM format"""

    gt_file = "Datasets/Replica/room0/traj.txt"
    output_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt"

    print(f"Converting {gt_file} to TUM format...")

    poses = []
    with open(gt_file, 'r') as f:
        lines = f.readlines()

    # Each pose is a 4x4 matrix stored as 16 values in 4 lines
    for i in range(0, len(lines), 4):
        if i + 3 < len(lines):
            # Read 4 lines to form a 4x4 matrix
            matrix = []
            for j in range(4):
                row = [float(x) for x in lines[i + j].strip().split()]
                if len(row) == 4:
                    matrix.append(row)

            if len(matrix) == 4:
                poses.append(np.array(matrix))

    print(f"Found {len(poses)} poses in ground truth")

    # Convert to TUM format
    with open(output_file, 'w') as f:
        for i, pose in enumerate(poses):
            # Extract translation
            translation = pose[:3, 3]

            # Extract rotation matrix and convert to quaternion
            rotation_matrix = pose[:3, :3]
            r = Rotation.from_matrix(rotation_matrix)
            quat = r.as_quat()  # [x, y, z, w] format

            # Write in TUM format: timestamp tx ty tz qx qy qz qw
            line = f"{i:06d} {translation[0]:.6f} {translation[1]:.6f} {translation[2]:.6f} {quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}"
            f.write(line + "\n")

    print(f"Converted ground truth saved to: {output_file}")
    return output_file

def align_trajectories():
    """Align estimated and ground truth trajectories to have the same number of poses"""

    gt_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt"
    est_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt"

    # Read both trajectories
    gt_poses = []
    with open(gt_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 8:
                gt_poses.append(line.strip())

    est_poses = []
    with open(est_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 8:
                est_poses.append(line.strip())

    print(f"Ground truth poses: {len(gt_poses)}")
    print(f"Estimated poses: {len(est_poses)}")

    # Align to the shorter trajectory
    min_poses = min(len(gt_poses), len(est_poses))
    print(f"Aligning to {min_poses} poses")

    # Create aligned files
    gt_aligned = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt"
    est_aligned = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt"

    with open(gt_aligned, 'w') as f:
        for i in range(min_poses):
            f.write(gt_poses[i] + "\n")

    with open(est_aligned, 'w') as f:
        for i in range(min_poses):
            f.write(est_poses[i] + "\n")

    print(f"Aligned trajectories saved:")
    print(f"  Ground truth: {gt_aligned}")
    print(f"  Estimated: {est_aligned}")

    return gt_aligned, est_aligned

def validate_tum_format(filename):
    """Validate TUM format"""
    print(f"\nValidating {filename}:")

    with open(filename, 'r') as f:
        lines = f.readlines()

    valid_lines = 0
    for i, line in enumerate(lines[:5]):  # Check first 5 lines
        parts = line.strip().split()
        if len(parts) == 8:
            valid_lines += 1
            print(f"  Line {i+1}: {' '.join(parts)}")
        else:
            print(f"  Line {i+1}: INVALID - {len(parts)} parts")

    print(f"  Total lines: {len(lines)}")
    print(f"  Format valid: {valid_lines == min(5, len(lines))}")

if __name__ == "__main__":
    # Convert ground truth
    gt_tum_file = convert_replica_to_tum()

    # Validate formats
    validate_tum_format(gt_tum_file)
    validate_tum_format("/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt")

    # Align trajectories
    gt_aligned, est_aligned = align_trajectories()

    print("\n" + "="*60)
    print("READY FOR ATE EVALUATION!")
    print("="*60)
    print(f"Run: !evo_ape tum {gt_aligned} {est_aligned} --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip")

Converting Datasets/Replica/room0/traj.txt to TUM format...
Found 0 poses in ground truth
Converted ground truth saved to: /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt

Validating /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt:
  Total lines: 0
  Format valid: True

Validating /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt:
  Line 1: 000000 3.452987 0.454611 0.593629 0.299185 0.417108 0.697356 0.500203
  Line 2: 000001 3.458086 0.458754 0.589913 0.301152 0.413567 0.695761 0.504169
  Line 3: 000002 3.463855 0.465824 0.588772 0.302009 0.410939 0.694982 0.506873
  Line 4: 000003 3.467487 0.479921 0.589067 0.302542 0.409459 0.694072 0.508995
  Line 5: 000004 3.471281 0.495672 0.591193 0.302290 0.408571 0.694056 0.509879
  Total lines: 501
  Format valid: True
Ground truth poses: 0
Estimated poses: 501
Aligning to 0 poses
Aligned trajectories saved:
  Ground truth: /content/drive/MyDrive/ESLAM_outputs/room0_

In [ ]:
# Save the conversion script to a file
%%writefile convert_replica_trajectory.py
import numpy as np
from scipy.spatial.transform import Rotation

def convert_replica_to_tum():
    """Convert Replica ground truth trajectory from 4x4 matrices to TUM format"""

    gt_file = "Datasets/Replica/room0/traj.txt"
    output_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt"

    print(f"Converting {gt_file} to TUM format...")

    poses = []
    with open(gt_file, 'r') as f:
        lines = f.readlines()

    # Each pose is a 4x4 matrix stored as 16 values in 4 lines
    for i in range(0, len(lines), 4):
        if i + 3 < len(lines):
            # Read 4 lines to form a 4x4 matrix
            matrix = []
            for j in range(4):
                row = [float(x) for x in lines[i + j].strip().split()]
                if len(row) == 4:
                    matrix.append(row)

            if len(matrix) == 4:
                poses.append(np.array(matrix))

    print(f"Found {len(poses)} poses in ground truth")

    # Convert to TUM format
    with open(output_file, 'w') as f:
        for i, pose in enumerate(poses):
            # Extract translation
            translation = pose[:3, 3]

            # Extract rotation matrix and convert to quaternion
            rotation_matrix = pose[:3, :3]
            r = Rotation.from_matrix(rotation_matrix)
            quat = r.as_quat()  # [x, y, z, w] format

            # Write in TUM format: timestamp tx ty tz qx qy qz qw
            line = f"{i:06d} {translation[0]:.6f} {translation[1]:.6f} {translation[2]:.6f} {quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}"
            f.write(line + "\n")

    print(f"Converted ground truth saved to: {output_file}")
    return output_file

def align_trajectories():
    """Align estimated and ground truth trajectories to have the same number of poses"""

    gt_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt"
    est_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt"

    # Read both trajectories
    gt_poses = []
    with open(gt_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 8:
                gt_poses.append(line.strip())

    est_poses = []
    with open(est_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 8:
                est_poses.append(line.strip())

    print(f"Ground truth poses: {len(gt_poses)}")
    print(f"Estimated poses: {len(est_poses)}")

    # Align to the shorter trajectory
    min_poses = min(len(gt_poses), len(est_poses))
    print(f"Aligning to {min_poses} poses")

    # Create aligned files
    gt_aligned = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt"
    est_aligned = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt"

    with open(gt_aligned, 'w') as f:
        for i in range(min_poses):
            f.write(gt_poses[i] + "\n")

    with open(est_aligned, 'w') as f:
        for i in range(min_poses):
            f.write(est_poses[i] + "\n")

    print(f"Aligned trajectories saved:")
    print(f"  Ground truth: {gt_aligned}")
    print(f"  Estimated: {est_aligned}")

    return gt_aligned, est_aligned

def validate_tum_format(filename):
    """Validate TUM format"""
    print(f"\nValidating {filename}:")

    with open(filename, 'r') as f:
        lines = f.readlines()

    valid_lines = 0
    for i, line in enumerate(lines[:5]):  # Check first 5 lines
        parts = line.strip().split()
        if len(parts) == 8:
            valid_lines += 1
            print(f"  Line {i+1}: {' '.join(parts)}")
        else:
            print(f"  Line {i+1}: INVALID - {len(parts)} parts")

    print(f"  Total lines: {len(lines)}")
    print(f"  Format valid: {valid_lines == min(5, len(lines))}")

if __name__ == "__main__":
    # Convert ground truth
    gt_tum_file = convert_replica_to_tum()

    # Validate formats
    validate_tum_format(gt_tum_file)
    validate_tum_format("/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt")

    # Align trajectories
    gt_aligned, est_aligned = align_trajectories()

    print("\n" + "="*60)
    print("READY FOR ATE EVALUATION!")
    print("="*60)
    print(f"Run: !evo_ape tum {gt_aligned} {est_aligned} --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip")

Writing convert_replica_trajectory.py


In [ ]:
!python convert_replica_trajectory.py

Converting Datasets/Replica/room0/traj.txt to TUM format...
Found 0 poses in ground truth
Converted ground truth saved to: /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt

Validating /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt:
  Total lines: 0
  Format valid: True

Validating /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt:
  Line 1: 000000 3.452987 0.454611 0.593629 0.299185 0.417108 0.697356 0.500203
  Line 2: 000001 3.458086 0.458754 0.589913 0.301152 0.413567 0.695761 0.504169
  Line 3: 000002 3.463855 0.465824 0.588772 0.302009 0.410939 0.694982 0.506873
  Line 4: 000003 3.467487 0.479921 0.589067 0.302542 0.409459 0.694072 0.508995
  Line 5: 000004 3.471281 0.495672 0.591193 0.302290 0.408571 0.694056 0.509879
  Total lines: 501
  Format valid: True
Ground truth poses: 0
Estimated poses: 501
Aligning to 0 poses
Aligned trajectories saved:
  Ground truth: /content/drive/MyDrive/ESLAM_outputs/room0_

In [ ]:
# Run ATE evaluation with aligned trajectories
!evo_ape tum /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip

[ERROR] TUM trajectory files must have 8 entries per row and no trailing delimiter at the end of the rows (space)


In [ ]:
# Let's first check what's actually in our files
!head -3 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt
!echo "---"
!head -3 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt 2>/dev/null || echo "GT file not found"

000000 3.452987 0.454611 0.593629 0.299185 0.417108 0.697356 0.500203
000001 3.458086 0.458754 0.589913 0.301152 0.413567 0.695761 0.504169
000002 3.463855 0.465824 0.588772 0.302009 0.410939 0.694982 0.506873
---


In [ ]:
# Fix the spacing issue in the trajectory file
%%writefile fix_spacing.py
def fix_trajectory_spacing():
    """Fix spacing in trajectory file"""

    input_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/estimated_trajectory.txt"
    output_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_fixed.txt"

    fixed_lines = []
    with open(input_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                # The line format appears to be: "000000 3.452987 0.454611..."
                # but the timestamp and first number might be concatenated

                # Extract timestamp (first 6 digits)
                timestamp = line[:6]
                rest = line[6:].strip()

                # Split the rest into coordinates
                coords = rest.split()

                if len(coords) == 7:  # tx ty tz qx qy qz qw
                    # Create properly spaced line
                    fixed_line = f"{timestamp} {' '.join(coords)}"
                    fixed_lines.append(fixed_line)
                    print(f"Fixed: {fixed_line}")
                else:
                    print(f"Warning: Expected 7 coordinates, got {len(coords)}: {line}")

    # Write fixed file
    with open(output_file, 'w') as f:
        for line in fixed_lines:
            f.write(line + '\n')

    print(f"\nFixed {len(fixed_lines)} lines and saved to {output_file}")

    # Validate the fixed file
    print("\nValidating fixed file:")
    with open(output_file, 'r') as f:
        for i, line in enumerate(f):
            if i < 3:  # Show first 3 lines
                parts = line.strip().split()
                print(f"Line {i+1}: {len(parts)} parts -> {line.strip()}")

if __name__ == "__main__":
    fix_trajectory_spacing()

Writing fix_spacing.py


In [ ]:
!python fix_spacing.py

Fixed: 000000 3.452987 0.454611 0.593629 0.299185 0.417108 0.697356 0.500203
Fixed: 000001 3.458086 0.458754 0.589913 0.301152 0.413567 0.695761 0.504169
Fixed: 000002 3.463855 0.465824 0.588772 0.302009 0.410939 0.694982 0.506873
Fixed: 000003 3.467487 0.479921 0.589067 0.302542 0.409459 0.694072 0.508995
Fixed: 000004 3.471281 0.495672 0.591193 0.302290 0.408571 0.694056 0.509879
Fixed: 000005 3.475099 0.510732 0.593311 0.302440 0.408111 0.693827 0.510469
Fixed: 000006 3.478410 0.524948 0.594183 0.302434 0.408232 0.693600 0.510685
Fixed: 000007 3.481829 0.537725 0.594680 0.302637 0.408794 0.693548 0.510185
Fixed: 000008 3.485704 0.547954 0.593954 0.303376 0.409466 0.693143 0.509758
Fixed: 000009 3.488510 0.557980 0.593080 0.303588 0.410875 0.693058 0.508613
Fixed: 000010 3.492729 0.570394 0.590849 0.304420 0.413030 0.692823 0.506686
Fixed: 000011 3.497056 0.582164 0.583269 0.305112 0.416313 0.692124 0.504535
Fixed: 000012 3.503844 0.592562 0.575594 0.306463 0.420063 0.691167 0.501912

In [ ]:
# Check the exact format of our files
!echo "=== ESTIMATED (first 3 lines) ==="
!head -3 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_fixed.txt
!echo "=== GROUND TRUTH (first 3 lines) ==="
!head -3 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt

# Count columns in each file
!echo "=== COLUMN COUNTS ==="
!head -1 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_fixed.txt | awk '{print "EST columns:", NF}'
!head -1 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt | awk '{print "GT columns:", NF}'

=== ESTIMATED (first 3 lines) ===
000000 3.452987 0.454611 0.593629 0.299185 0.417108 0.697356 0.500203
000001 3.458086 0.458754 0.589913 0.301152 0.413567 0.695761 0.504169
000002 3.463855 0.465824 0.588772 0.302009 0.410939 0.694982 0.506873
=== GROUND TRUTH (first 3 lines) ===
=== COLUMN COUNTS ===
EST columns: 8


In [ ]:
# Check if ground truth file exists and debug the conversion
!ls -la /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt 2>/dev/null || echo "GT file missing"

# Check the original ground truth format again
!echo "Original GT format:"
!head -8 Datasets/Replica/room0/traj.txt

-rw------- 1 root root 0 Jun 19 22:12 /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt
Original GT format:
-3.205696220032456800e-01 4.480551946958012954e-01 -8.345547674986967257e-01 3.452987416406734678e+00 9.472249560947475500e-01 1.516354520391845484e-01 -2.824386167580063001e-01 4.546110134135947223e-01 1.078977932490423164e-16 -8.810523436158487209e-01 -4.730187816662466682e-01 5.936285447159415085e-01 0.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e+00
-3.139329310152697561e-01 4.530657137828867920e-01 -8.343725629588769621e-01 3.457151553373337016e+00 9.494451615676188228e-01 1.498056477907042938e-01 -2.758843110179438618e-01 4.697096078660035756e-01 1.076219806983754285e-16 -8.788001632250705963e-01 -4.771899759169185296e-01 5.942743770991206764e-01 0.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00 1.000000000000000000e+00
-3.084751309215140735e-01 4.567669288580877285e-01 -8.34390

In [ ]:
# Create a working ground truth converter
%%writefile convert_gt_working.py
import numpy as np
from scipy.spatial.transform import Rotation

def parse_replica_gt():
    """Parse Replica ground truth format correctly"""

    gt_file = "Datasets/Replica/room0/traj.txt"
    output_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt"

    print(f"Converting {gt_file}...")

    poses = []
    with open(gt_file, 'r') as f:
        for line_num, line in enumerate(f):
            # Each line contains 16 values representing a 4x4 matrix
            values = line.strip().split()

            if len(values) == 16:
                # Convert to floats and reshape to 4x4 matrix
                try:
                    matrix_values = [float(v) for v in values]
                    matrix = np.array(matrix_values).reshape(4, 4)
                    poses.append(matrix)

                    if line_num < 3:  # Debug first few
                        print(f"Line {line_num}: Matrix shape {matrix.shape}")
                        print(f"  Translation: {matrix[:3, 3]}")

                except ValueError as e:
                    print(f"Error on line {line_num}: {e}")
            else:
                print(f"Line {line_num}: Expected 16 values, got {len(values)}")
                if line_num < 5:  # Show first few problematic lines
                    print(f"  Content: {line.strip()[:100]}...")

    print(f"Successfully parsed {len(poses)} poses")

    if len(poses) == 0:
        print("ERROR: No poses found!")
        return False

    # Convert to TUM format
    with open(output_file, 'w') as f:
        for i, pose in enumerate(poses):
            # Extract translation (last column, first 3 rows)
            translation = pose[:3, 3]

            # Extract rotation matrix (top-left 3x3)
            rotation_matrix = pose[:3, :3]

            # Convert rotation matrix to quaternion
            r = Rotation.from_matrix(rotation_matrix)
            quat = r.as_quat()  # [x, y, z, w] format

            # Write in TUM format: timestamp tx ty tz qx qy qz qw
            line = f"{i:06d} {translation[0]:.6f} {translation[1]:.6f} {translation[2]:.6f} {quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}"
            f.write(line + '\n')

    print(f"Saved {len(poses)} poses to {output_file}")

    # Validate output
    print("\nFirst 3 converted poses:")
    with open(output_file, 'r') as f:
        for i, line in enumerate(f):
            if i < 3:
                parts = line.strip().split()
                print(f"  Line {i+1}: {len(parts)} parts -> {line.strip()}")

    return True

if __name__ == "__main__":
    success = parse_replica_gt()
    if success:
        print("\nGround truth conversion successful!")
    else:
        print("\nGround truth conversion failed!")

Writing convert_gt_working.py


In [ ]:
!python convert_gt_working.py

Converting Datasets/Replica/room0/traj.txt...
Line 0: Matrix shape (4, 4)
  Translation: [3.45298742 0.45461101 0.59362854]
Line 1: Matrix shape (4, 4)
  Translation: [3.45715155 0.46970961 0.59427438]
Line 2: Matrix shape (4, 4)
  Translation: [3.46139443 0.48418577 0.59430219]
Successfully parsed 2000 poses
Saved 2000 poses to /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt

First 3 converted poses:
  Line 1: 8 parts -> 000000 3.452987 0.454611 0.593629 0.500203 0.697356 -0.417108 -0.299185
  Line 2: 8 parts -> 000001 3.457152 0.469710 0.594274 0.503352 0.696586 -0.414408 -0.299451
  Line 3: 8 parts -> 000002 3.461394 0.484186 0.594302 0.505862 0.695842 -0.412361 -0.299777

Ground truth conversion successful!


In [ ]:
# Align to have same number of poses (limit to 500 since we only processed 500 frames)
%%writefile align_and_evaluate.py
def align_trajectories():
    """Align trajectories to same length"""

    gt_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ground_truth_tum.txt"
    est_file = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_fixed.txt"

    # Read trajectories
    with open(gt_file, 'r') as f:
        gt_lines = f.readlines()

    with open(est_file, 'r') as f:
        est_lines = f.readlines()

    print(f"Ground truth poses: {len(gt_lines)}")
    print(f"Estimated poses: {len(est_lines)}")

    # Align to minimum length
    min_poses = min(len(gt_lines), len(est_lines))
    print(f"Aligning to {min_poses} poses")

    # Save aligned files
    gt_aligned = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt"
    est_aligned = "/content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt"

    with open(gt_aligned, 'w') as f:
        for i in range(min_poses):
            f.write(gt_lines[i])

    with open(est_aligned, 'w') as f:
        for i in range(min_poses):
            f.write(est_lines[i])

    print(f"Saved aligned trajectories")
    print(f"  GT: {gt_aligned}")
    print(f"  EST: {est_aligned}")

    return gt_aligned, est_aligned

if __name__ == "__main__":
    gt_file, est_file = align_trajectories()
    print(f"\nRun ATE evaluation:")
    print(f"!evo_ape tum {gt_file} {est_file} --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip")

Writing align_and_evaluate.py


In [ ]:
!python align_and_evaluate.py

Ground truth poses: 2000
Estimated poses: 501
Aligning to 501 poses
Saved aligned trajectories
  GT: /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt
  EST: /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt

Run ATE evaluation:
!evo_ape tum /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip


In [ ]:
!evo_ape tum /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/ate_results.zip

APE w.r.t. translation part (m)
(not aligned)

       max	1.589668
      mean	0.737848
    median	0.784924
       min	0.000000
      rmse	0.813709
       sse	331.723411
       std	0.343079



In [ ]:
# Get additional trajectory metrics
!evo_rpe tum /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/gt_aligned.txt /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/est_aligned.txt --plot --save_results /content/drive/MyDrive/ESLAM_outputs/room0_eval_500/rpe_results.zip

RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(not aligned)

       max	0.062191
      mean	0.023902
    median	0.021448
       min	0.001955
      rmse	0.026455
       sse	0.349931
       std	0.011339



In [ ]:
import psutil

In [ ]:
# Check GPU memory usage
!nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader,nounits

# Check system memory
import psutil
memory_info = psutil.virtual_memory()
print(f"System RAM usage: {memory_info.used / (1024**3):.2f} GB / {memory_info.total / (1024**3):.2f} GB")
print(f"Memory utilization: {memory_info.percent:.1f}%")

5, 40960
System RAM usage: 1.67 GB / 83.48 GB
Memory utilization: 3.0%


In [ ]:
# Calculate runtime metrics from your run
frames_processed = 501
# You can estimate total runtime from when you started to when it finished
# Let's say it took approximately X minutes (you observed this)

# Calculate performance metrics
print("=== RUNTIME PERFORMANCE ===")
print(f"Total frames processed: {frames_processed}")
print(f"Target frames: 500")
print(f"Processing efficiency: {500/frames_processed*100:.1f}%")

# You can report approximate values like:
estimated_total_time_minutes = 15
fps = frames_processed / (estimated_total_time_minutes * 60)
time_per_frame = (estimated_total_time_minutes * 60) / frames_processed

print(f"Estimated total runtime: ~{estimated_total_time_minutes} minutes")
print(f"Processing speed: ~{fps:.2f} FPS")
print(f"Time per frame: ~{time_per_frame:.2f} seconds/frame")

=== RUNTIME PERFORMANCE ===
Total frames processed: 501
Target frames: 500
Processing efficiency: 99.8%
Estimated total runtime: ~15 minutes
Processing speed: ~0.56 FPS
Time per frame: ~1.80 seconds/frame
